In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Regression').getOrCreate()

In [4]:
df_pyspark = spark.read.csv('MLib_Values.csv', header=True, inferSchema=True)

In [5]:
df_pyspark.show()

+---------+---+----------+------+
|     Name|Age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [6]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [7]:
df_pyspark.columns

['Name', 'Age', 'Experience', 'Salary']

In [ ]:
#Vector Assembler
#[Age, Experience] -----> new feature -----> independent feature

In [8]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols= ["age", "Experience"], outputCol="Independent Features")

In [9]:
output = featureassembler.transform(df_pyspark)

In [10]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|Age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [11]:
type(output)

pyspark.sql.classic.dataframe.DataFrame

In [12]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [14]:
finalized_output = output.select("Independent Features", "Salary")

In [15]:
finalized_output.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [17]:
from pyspark.ml.regression import LinearRegression

#train test split
train_data, test_data = finalized_output.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

In [18]:
regressor.coefficients

DenseVector([-383.9733, 1711.1853])

In [19]:
regressor.intercept

23998.330550919385

In [20]:
pred_results = regressor.evaluate(test_data)

In [21]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000| 17646.07679465793|
|          [23.0,2.0]| 18000|18589.315525876566|
+--------------------+------+------------------+



In [22]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(1617.6961602672473, 3674507.5961331767)